In [ ]:
import plotly.graph_objects as go
import numpy as np
import random
import copy
import time

N = 100
cities = [[random.random(), random.random(), random.random()] for _ in range(N)]

## **Class implementation**

In [ ]:
class AG_TSP_3D():
    def __init__(self, alphabet: list[list]):
        """Create the population"""

        self.population = []
        self.fitnesses = []
        for i in range(N):
            self.population.append(copy.copy(alphabet))
            random.shuffle(self.population[i])

    def sort_pop(self, fitness_function, reverse_sort: bool) -> tuple[list[list], list]:
        """Sort population by fitness function. Return tuple with population list and fitness list"""

        fitness_list = [fitness_function(ind) for ind in self.population]
        lista = sorted(zip(self.population, fitness_list), key=lambda x: x[1], reverse=reverse_sort)
        self.population = [x[0] for x in lista]
        self.fitnesses = [x[1] for x in lista]
    
    def select(self, T: int) -> list[list]:
        """Return a copy of an indivudual by tournament selection. Population already ordered by fitness"""

        choices=random.choices(copy.copy(self.population), k=T)
        indices=[self.population.index(c) for c in choices]
        return self.population[np.argmin(indices)]
    
    def crossover(self, parent1: list, parent2: list, pcross: float) -> tuple[list, list]:
        """Performs crossover on two parents to generate two children. PMX method"""

        child1 = parent1[:]
        child2 = parent2[:]

        if random.random() < pcross:
            size = len(parent1)
            start, end = sorted(random.sample(range(size), 2))

            # Copiar segmento intermedio
            child1[start:end+1] = parent1[start:end+1]
            child2[start:end+1] = parent2[start:end+1]

            # Mapear elementos del segmento intermedio para el primer hijo
            for i in range(start, end+1):
                if parent2[i] not in child1:
                    pos = i
                    while start <= pos <= end:
                        pos = parent2.index(parent1[pos])
                    child1[pos] = parent2[i]

            # Mapear elementos del segmento intermedio para el segundo hijo
            for i in range(start, end+1):
                if parent1[i] not in child2:
                    pos = i
                    while start <= pos <= end:
                        pos = parent1.index(parent2[pos])
                    child2[pos] = parent1[i]

            # Rellenar valores restantes en el primer hijo
            for i in range(size):
                if child1[i] == -1:
                    child1[i] = parent2[i]

            # Rellenar valores restantes en el segundo hijo
            for i in range(size):
                if child2[i] == -1:
                    child2[i] = parent1[i]

        return child1, child2

    def mutate(self, individual: list[list], pmut: float) -> list:
        """Mutate an individual, swap elements. Return mutated individual"""
    
        def mutate_swap(individual: list[list], pmut: float) -> list[list]:
            if random.random() < pmut:
                index1, index2 = random.choices(range(len(individual)), k=2)
                individual[index1], individual[index2] = individual[index2], individual[index1]
            return individual

        def mutate_insert(individual: list[list], pmut: float) -> list[list]:
            if random.random() < pmut:
                index_pop, index_insert = random.choices(range(len(individual)), k=2)
                value = individual.pop(index_pop)
                individual.insert(index_insert, value)
            return individual

        def mutate_reverse(individual: list[list], pmut: float) -> list[list]: 
            if random.random() < pmut:
                index1, index2 = sorted(random.sample(range(len(individual)), 2))
                individual[index1:index2 + 1] = individual[index1:index2 + 1][::-1]
            return individual
        
        mutations = [mutate_swap, mutate_insert, mutate_reverse]
        operator = random.choice(mutations)
        return operator(individual, pmut)
    

    def evolve(self, fitness_function, pmut=0.1, pcross=0.7, ngen=100, T=2, trace=50, reverse_sort=False, elitism=False) -> None:
        """Evolution procedure"""

        def checkvalid(ind: list[list]):
            """Compare the length of the set with the original array"""

            tuple_subarrays = [tuple(subarray) for subarray in ind]
            return len(tuple_subarrays) == len(set(tuple_subarrays))
        
        for i in range(ngen):
            new_pop = []
            self.sort_pop(fitness_function, reverse_sort)
            if elitism:
                new_pop.append(self.population[0])
                new_pop.append(self.population[1])
            while len(new_pop) != 100:   
                individual1 = self.select(T)
                individual2 = self.select(T)
                child1,child2 = self.crossover(individual1, individual2, pcross)
                if checkvalid(child1) and checkvalid(child2):
                    mutated1 = self.mutate(child1, pmut)
                    mutated2 = self.mutate(child2, pmut)
                    new_pop.append(mutated1)
                    new_pop.append(mutated2)
            
            if i % trace == 0 or i == ngen-1: # en la última gen se ordena
                self.sort_pop(fitness_function, reverse_sort)
                print(f"Num gen: {i}, Fitness óptimo: {self.fitnesses[0]}")
            self.population = [*new_pop] # make a copy
        return self.population

In [6]:
def dist(x, y):
    return ((x[0] - y[0]) ** 2 + (x[1] - y[1]) ** 2 + (x[2] - y[2]) ** 2) ** 0.5

def globaldist(cities): 
	acum = 0.0
	for i in range(len(cities)):
		acum += dist(cities[i-1],cities[i]) # -1 is the last element
	return acum

def fit_cities(cities): # fitness para TSP
	return 1 / (1+globaldist(cities))

## **Visual test**

In [7]:
def draw(solution: list[list]):
    bucle = solution + [solution[0]]  # Crear el bucle que cierra la ruta
    c = np.array(bucle)
    x = c[:, 0]
    y = c[:, 1]
    z = c[:, 2]

    dist = globaldist(bucle)
    title = 'Length=%5.3f' % dist

    fig = go.Figure()
    fig.add_trace(go.Scatter3d(x=x, y=y, z=z, mode='lines', line=dict(color='green', width=4), name='Ruta'))
    fig.add_trace(go.Scatter3d(x=[x[0]], y=[y[0]], z=[z[0]],    mode='markers', marker=dict(color='blue', size=10), name='Primera ciudad'))
    fig.add_trace(go.Scatter3d(x=x[1:-1], y=y[1:-1], z=z[1:-1], mode='markers', marker=dict(color='red', size=6), name='Resto ciudades'))

    fig.update_layout(
        title=title,
        scene=dict(
            xaxis_title='X',
            yaxis_title='Y',
            zaxis_title='Z'
        ),
        width=700,  # Ancho de la figura
        height=550,  # Alto de la figura
        showlegend=True
    )
    fig.show()

draw(cities)

## **---------------------------Tests---------------------------**

In [8]:
start = time.time()
genetic_algorithm = AG_TSP_3D(cities) 
final_population = genetic_algorithm.evolve(fitness_function=globaldist, pmut=0.1, ngen=10000, T=6, trace=500)
minutos, segundos = divmod(time.time()-start, 60)
print(f"*******Tiempo transcurrido: {int(minutos)} minutos y {segundos:.2f} segundos*******")

draw(final_population[0])

Num gen: 0, Fitness óptimo: 59.37069074053992
Num gen: 500, Fitness óptimo: 24.616016585807532
Num gen: 1000, Fitness óptimo: 21.946223504988613
Num gen: 1500, Fitness óptimo: 20.952454603962288
Num gen: 2000, Fitness óptimo: 19.495102044796873
Num gen: 2500, Fitness óptimo: 19.416984041644202
Num gen: 3000, Fitness óptimo: 18.98074581764095
Num gen: 3500, Fitness óptimo: 18.98074581764095
Num gen: 4000, Fitness óptimo: 18.98074581764095
Num gen: 4500, Fitness óptimo: 18.98074581764095
Num gen: 5000, Fitness óptimo: 18.81812169780553
Num gen: 5500, Fitness óptimo: 18.792433511084717
Num gen: 6000, Fitness óptimo: 18.792433511084713
Num gen: 6500, Fitness óptimo: 18.760451674764408
Num gen: 7000, Fitness óptimo: 18.707137741001404
Num gen: 7500, Fitness óptimo: 18.596852271878237
Num gen: 8000, Fitness óptimo: 18.596852271878237
Num gen: 8500, Fitness óptimo: 18.596852271878237
Num gen: 9000, Fitness óptimo: 18.596852271878237
Num gen: 9500, Fitness óptimo: 18.596852271878237
Num gen: 9